# Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import joblib

from sklearn.linear_model import Lasso, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


In [5]:
data = pd.read_csv(r'C:\64KTPM3\HocMay_Demo1\data\data_n7.csv')

In [6]:
print(data.shape)
print(data.columns)
data.head()

(373, 6)
Index(['Age', 'Gender', 'Education Level', 'Job Title', 'Years of Experience',
       'Salary'],
      dtype='object')


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32,Male,Bachelor's,Software Engineer,5.0,90000
1,28,Female,Master's,Data Analyst,3.0,65000
2,45,Male,PhD,Senior Manager,15.0,150000
3,36,Female,Bachelor's,Sales Associate,7.0,60000
4,52,Male,Master's,Director,20.0,200000


# 1. Tiền xử lý dữ liệu

### 1.1 Loại bỏ các cột không cần thiết 

In [7]:

data = data.drop(columns=['Gender', 'Job Title'])


In [5]:
print(data.shape)
data.head()     

(373, 4)


,Age,Education Level,Years of Experience,Salary
0,32,Bachelor's,5.0,90000
1,28,Master's,3.0,65000
2,45,PhD,15.0,150000
3,36,Bachelor's,7.0,60000
4,52,Master's,20.0,200000


### 1.2 Mã hóa 

In [8]:
from sklearn.preprocessing import OrdinalEncoder 

# Bachelor's < Master's < PhD => Bachelor's = 0; Master's = 1; PhD = 2
Encoder = OrdinalEncoder(categories=[["Bachelor's", "Master's", "PhD"]]) 
                                  
data['Education Level'] = Encoder.fit_transform(data[['Education Level']])

In [7]:
data.head()

,Age,Education Level,Years of Experience,Salary
0,32,0.0,5.0,90000
1,28,1.0,3.0,65000
2,45,2.0,15.0,150000
3,36,0.0,7.0,60000
4,52,1.0,20.0,200000


# 2. Huấn luyện mô hình

### 2.1 Hồi quy tuyến tính

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

###  Tách dữ liệu thành biến đầu vào (X) và đầu ra (y)

In [11]:
X = data.drop('Salary', axis=1)  # Tập dữ liệu đầu vào
y = data['Salary']               # Tập dữ liệu đầu ra

In [12]:
X.head() 

,Age,Education Level,Years of Experience
0,32,0.0,5.0
1,28,1.0,3.0
2,45,2.0,15.0
3,36,0.0,7.0
4,52,1.0,20.0


In [13]:
y.head()

0     90000
1     65000
2    150000
3     60000
4    200000
Name: Salary, dtype: int64

#### Chia dữ liệu thành 80% cho tập huấn luyện và 20% cho tập kiểm tra

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Tạo mô hình hồi quy tuyến tính

In [15]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

# Huấn luyện mô hình
linear_model.fit(X_train, y_train)

LinearRegression()

In [16]:

# Dự đoán trên tập kiểm tra
y_pred_linear = linear_model.predict(X_test)

# Đánh giá mô hình
mse_linear = mean_squared_error(y_test, y_pred_linear)  # Mean Squared Error
r2_linear = r2_score(y_test, y_pred_linear)             # R-squared

# Input
print(f"Mean Squared Error (Linear Regression): {mse_linear}")
print(f"R-squared (Linear Regression): {r2_linear}")

# So sánh giá trị thực tế và giá trị dự đoán
comparison_df_linear = pd.DataFrame({'Thực tế': y_test, 'Dự đoán': y_pred_linear})
comparison_df_linear = comparison_df_linear.round(4)  
print(comparison_df_linear.head(10))  




Mean Squared Error (Linear Regression): 262458894.43150213
R-squared (Linear Regression): 0.8905318482933029
     Thực tế      Dự đoán
327   180000  167511.3137
33     65000   97167.8380
15    125000  127812.0981
314    80000   86096.1167
57    140000  156439.5925
239   160000  162080.8077
76    160000  161659.3894
119   120000   99051.8839
305    50000   54476.3003
126    95000  103019.7624


In [17]:
import joblib

# Huấn luyện mô hình Linear Regression
linear_model.fit(X_train, y_train)

# Lưu mô hình Linear Regression
joblib.dump(linear_model, 'linear_model.pkl')


['linear_model.pkl']

### 2.2 Mô hình Lasso 

#### Huấn luyện mô hình

In [18]:
# Tạo mô hình hồi quy Lasso
lasso_model = Lasso(alpha=3000)  
# Huấn luyện mô hình
lasso_model.fit(X_train, y_train)

# Lưu mô hình
joblib.dump(lasso_model, 'lasso_model.pkl')


['lasso_model.pkl']

In [19]:
# Tải mô hình đã lưu
lasso_model = joblib.load('lasso_model.pkl')

# Dự đoán với mô hình đã tải
y_pred = lasso_model.predict(X_test)


In [20]:
# Dự đoán trên tập kiểm tra
y_pred_lasso = lasso_model.predict(X_test)

# Đánh giá mô hình
mse_lasso = mean_squared_error(y_test, y_pred_lasso)  
r2_lasso = r2_score(y_test, y_pred_lasso)            

# In kết quả
print(f"MSE (Lasso Regression): {mse_lasso}")
print(f"R2 (Lasso Regression): {r2_lasso}")

# So sánh giá trị thực tế và giá trị dự đoán
comparison_df_lasso = pd.DataFrame({'Thực tế': y_test, 'Dự đoán': y_pred_lasso})
comparison_df_lasso = comparison_df_lasso.round(4) 
print(comparison_df_lasso.head(10)) 

MSE (Lasso Regression): 246745656.73251817
R2 (Lasso Regression): 0.8970856329991388
     Thực tế      Dự đoán
327   180000  169268.1294
33     65000  101381.5048
15    125000  135481.9127
314    80000   79550.0254
57    140000  147436.6499
239   160000  153616.3939
76    160000  172560.3768
119   120000   95334.8329
305    50000   55145.2406
126    95000   98089.2574


#### Mạng nơ ron

In [21]:
# Chia dữ liệu và chuẩn hóa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Tạo và huấn luyện mô hình
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=0)
mlp_model.fit(X_train_scaled, y_train)

# Dự đoán và đánh giá
y_pred_mlp = mlp_model.predict(X_test_scaled)
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print(f"Mean Squared Error (MLP Regressor): {mse_mlp}")
print(f"R-squared (MLP Regressor): {r2_mlp}")

# So sánh giá trị thực tế và dự đoán
comparison_df_mlp = pd.DataFrame({'Thực tế': y_test, 'Dự đoán': y_pred_mlp}).round(4)
print(comparison_df_mlp.head(10))


Mean Squared Error (MLP Regressor): 7277589640.889051
R-squared (MLP Regressor): -2.035386888272698
     Thực tế     Dự đoán
327   180000  49506.9044
33     65000  13378.5639
15    125000  21699.0693
314    80000  13530.4764
57    140000  50059.5892
239   160000  53090.1944
76    160000  39862.4043
119   120000  17045.5970
305    50000  20963.7323
126    95000  17627.6156


c:\64KTPM3\HocMay_Demo1\moiTruong\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


### Lưu mô hình mlp

In [18]:
joblib.dump(mlp_model, 'mlp_model.pkl')


['mlp_model.pkl']

### Stacking

In [22]:

# Tạo các mô hình cơ sở
base_models = [
    ('linear', LinearRegression()),
    ('lasso', Lasso(alpha=0.1)),
    ('mlp', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=0))
]

# Tạo mô hình meta
meta_model = LinearRegression()

# Tạo và huấn luyện mô hình stacking
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
stacking_model.fit(X_train_scaled, y_train)

# Dự đoán và đánh giá
y_pred_stacking = stacking_model.predict(X_test_scaled)
mse_stacking = mean_squared_error(y_test, y_pred_stacking)
r2_stacking = r2_score(y_test, y_pred_stacking)

print(f"Mean Squared Error (Stacking): {mse_stacking}")
print(f"R-squared (Stacking): {r2_stacking}")

# So sánh giá trị thực tế và dự đoán
comparison_df_stacking = pd.DataFrame({'Thực tế': y_test, 'Dự đoán': y_pred_stacking}).round(4)
print(comparison_df_stacking.head(10))


c:\64KTPM3\HocMay_Demo1\moiTruong\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\64KTPM3\HocMay_Demo1\moiTruong\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\64KTPM3\HocMay_Demo1\moiTruong\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\64KTPM3\HocMay_Demo1\moiTruong\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\64KTPM3\HocMa

Mean Squared Error (Stacking): 261076440.70277375
R-squared (Stacking): 0.8911084515546697
     Thực tế      Dự đoán
327   180000  166357.8358
33     65000   98967.4382
15    125000  129415.9029
314    80000   87516.1322
57    140000  154848.0451
239   160000  160231.2409
76    160000  161718.0392
119   120000  100434.8734
305    50000   53792.3385
126    95000  104394.5921


c:\64KTPM3\HocMay_Demo1\moiTruong\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


### Lưu mô hình stacking

In [20]:
joblib.dump(stacking_model, 'stacking_model.pkl')


['stacking_model.pkl']